In [1]:
import sys
import os

# Get the project root directory (parent of the notebooks folder)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the src folder to the Python path
sys.path.append(project_root)

In [2]:
from src.data_preprocessing import DataLoaders

loaders = DataLoaders(trial_data=False)
train_dataloader = loaders.get_train_dataloader()
test_dataloader = loaders.get_test_dataloader()
# for images, labels in train_dataloader:
#     print(f'Batch of images shape: {images.shape}')
#     print(f'Batch of labels: {labels}')

# for images, labels in test_dataloader:
#     print(f'Batch of images shape: {images.shape}')
#     print(f'Batch of labels: {labels}')

In [3]:
import torch


def evaluate(model, test_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [4]:

from vit_pytorch import SimpleViT
import torch.optim as optim
import torch.nn as nn
import json
import wandb
from src.models.tiny_vit import tiny_vit_5m_224

num_epochs = 10
learning_rate = 0.001
with open('../data/country_to_index_mapped.json', 'r') as f:
    num_classes = len(json.load(f))

model = tiny_vit_5m_224(pretrained=True)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device("mps")
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")
# Assuming parameters are of type float32 (4 bytes)
param_size_in_bytes = total_params * 4
# Convert to megabytes
param_size_in_mb = param_size_in_bytes / (1024 ** 2)
print(f"Model size: {param_size_in_mb:.2f} MB")

wandb.init(
    project="geoguessr AI",
    config={
    "learning_rate": learning_rate,
    "architecture": "TinyViT-5M-224",
    "dataset": "street-location-images- data-mapped",
    "epochs": num_epochs,
    "model_parameters": total_params,
    "model_size": param_size_in_mb
    }
)


ModuleNotFoundError: No module named 'models'

In [ ]:
from tqdm import tqdm

model.to(device)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch_idx, (inputs, labels) in enumerate(tqdm(train_dataloader)):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_dataloader)
    train_accuracy = 100.0 * correct_predictions / total_samples
    test_loss, test_accuracy = evaluate(model, test_dataloader, criterion, device)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%, '
          f'Test Loss: {test_loss:.4f}, '
          f'Test Accuracy: {test_accuracy:.2f}%')
    wandb.log({"train_loss": train_loss, "test_loss": test_loss, "train_accuracy": train_accuracy, "test_accuracy": test_accuracy})
wandb.finish()


In [ ]:
import torch
import torch
from transformers import ViTConfig
from huggingface_hub import login, Repository
from dotenv import load_dotenv
import os
import shutil

load_dotenv()


model_save_path = "../data/models/"
repo = Repository(local_dir=model_save_path, clone_from="mwitjez/geoguessr_tiny_ViT")

torch.save(model, f"{model_save_path}/geoguessr_model.bin")

config = ViTConfig()
config.save_pretrained(model_save_path)

repo.push_to_hub()
if os.path.exists(model_save_path):
    shutil.rmtree(model_save_path)
    print(f"'{model_save_path}' has been removed successfully.")
else:
    print(f"'{model_save_path}' does not exist.")
